## Tutorial
The basic idea is :


```
from ultralytics import YOLO

model = YOLO("yolov10s.pt") # We are going off the pretrained model, not starting from scratch

model.train(data="dataset_folder/data.yaml", epochs=1000, batch=-1, device=0, cache=True, classes=None, patience=50)
# A YOLO dataset is a folder with a data.yaml file giving the params to the model
# epochs = 1000 should ensure the model keeps on training until it judges itself to have attained its best possible performance
# batch = -1 allows the model engine to autodetermine the best batching size (parallelizing inference). Use 0.55 (55%) if VRAM runs out (lower if still runs out)
# device=0 ensures that the model runs on the first GPU available (if more than one : device=[0,1,...]
# device=cpu would allow training on cpu if no GPU available
# by default this script runs on first available gpu, defaults to cpu if none
#cache=True means the images are cached instead of loaded from disk on each pass (make sure enough RAM available)'
# Classes = None, means training on all classes, classes=list[int] will only train on the classes in the list

```

After this process, the trained model is at `runs/detect/train/weights/best.pt`




# Train YOLO Object Detection on a Custom Dataset

---


## Inputs and parameters

In [1]:
model_base = "yolo11m.pt"
export_formats = ["onnx", "openvino", "torchscript"]
export_zip_name = "trained_model_m_nounknown.zip"
training_params = {
    "epochs" : 1000,
    "batch" : -1,
    "cache" : True,
    "imgsz" : 640,
    "classes" : [0,1,2,3,4,5,6,7,8,9,11] #everything but unknown
}

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
dataset_zip = "/content/drive/MyDrive/Colab Notebooks/datasets/dataset.zip"

**ALWAYS GO CHECK THE PATH IN THE BEFORE LAST CELL, BY DEFAULT THIS WILL SAVE THE MODELS IN RUNS>DETECT>TRAIN2 !**

## Device setup

make sure we have access to GPU

In [4]:
from tensorflow.test import gpu_device_name

DEVICE = gpu_device_name()
if len(DEVICE) > 0:
  DEVICE = DEVICE[12:]
  ! nvidia-smi
else:
  DEVICE = "cpu"

Mon Feb 10 06:41:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             51W /  400W |     423MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLO


In [6]:
!pip install ultralytics

Check that YOLO runs and all required librairies are present

In [7]:
from IPython.display import display, Image, clear_output
clear_output()
!yolo check

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 33.2/112.6 GB disk)

OS                  Linux-6.1.85+-x86_64-with-glibc2.35
Environment         Colab
Python              3.11.11
Install             pip
RAM                 83.48 GB
Disk                33.2/112.6 GB
CPU                 Intel Xeon 2.20GHz
CPU count           12
GPU                 NVIDIA A100-SXM4-40GB, 40507MiB
GPU count           1
CUDA                12.4

numpy               ✅ 1.26.4<=2.1.1,>=1.23.0
matplotlib          ✅ 3.10.0>=3.3.0
opencv-python       ✅ 4.11.0.86>=4.6.0
pillow              ✅ 11.1.0>=7.1.2
pyyaml              ✅ 6.0.2>=5.3.1
requests            ✅ 2.32.3>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.5.1+cu124>=1.8.0
torch               ✅ 2.5.1+cu124!=2.4.0,>=1.8.0; sys_platform == "win32"
torchvision         ✅ 0.20.1+cu124>=0.9.0
tqdm                ✅ 4.67.1>=4.64.0
psutil              ✅ 5.9.5

In [8]:
#!pip install roboflow

#from roboflow import Roboflow
#rf = Roboflow(api_key="obcyD7henw5LjBBuINZP")
#project = rf.workspace("projet-industrie").project("traffic-counting-singapore")
#version = project.version(3)
#dataset = version.download("yolov11")

In [9]:
from ultralytics import YOLO
import wandb
wandb.init(mode="disabled")

## Training

### Open dataset

In [10]:
from ultralytics.utils.downloads import unzip_file

dataset_dir = unzip_file(dataset_zip, path="/content/datasets")
print(f'{dataset_dir=}')
files = os.listdir(dataset_dir)
print(f'{files=}')
dataset_yaml = next((f for f in files if f.endswith('.yaml')), None)
dataset_yaml = os.path.join(dataset_dir, dataset_yaml)
print(f'{dataset_yaml=}')

WARNING ⚠️ Skipping /content/drive/MyDrive/Colab Notebooks/datasets/dataset.zip unzip as destination directory /content/datasets/dataset is not empty.
dataset_dir=PosixPath('/content/datasets/dataset')
files=['val', 'test', 'data.yaml', 'train']
dataset_yaml='/content/datasets/dataset/data.yaml'


### Run training

In [11]:
model = YOLO(model_base)

results = model.train(data=dataset_yaml, epochs=training_params['epochs'], batch=training_params['batch'], device=DEVICE, cache=training_params['cache'], imgsz=training_params['imgsz'], classes=training_params['classes'])
for format in export_formats:
  model.export(format=format)

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/datasets/dataset/data.yaml, epochs=1000, time=None, patience=100, batch=0.55, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

train: Scanning /content/datasets/dataset/train/labels... 533 images, 0 backgrounds, 0 corrupt: 100%|██████████| 533/533 [00:00<00:00, 723.48it/s]

train: New cache created: /content/datasets/dataset/train/labels.cache


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.3GB RAM): 100%|██████████| 533/533 [00:00<00:00, 567.39it/s]


...


1000 epochs completed in 1.727 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 40.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 40.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11m summary (fused): 303 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


                   all        133       5342      0.927      0.784      0.863      0.696
               Bicycle         61         88      0.935       0.75      0.864      0.732
                   Bus         47        120      0.971       0.95      0.976      0.925
                   Car        110       2571      0.962      0.821      0.904      0.756
         Flatbed truck         64         66      0.968      0.909      0.958      0.722
               Minibus         89        318      0.936      0.823      0.899      0.764
            Motorcycle        130        453      0.924       0.72      0.814      0.615
            Pedestrian        106       1036      0.909      0.378       0.57      0.357
              Rickshaw          7          7      0.685      0.857      0.871      0.626
               Scooter         45        135      0.964      0.783      0.893      0.687
                 Truck         66        172      0.991      0.971      0.988      0.924
               Unknow

<frozen importlib.util>:262: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.


OpenVINO: export success ✅ 11.2s, saved as 'runs/detect/train2/weights/best_openvino_model/' (76.9 MB)

Export complete (12.2s)
Results saved to /content/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best_openvino_model imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train2/weights/best_openvino_model imgsz=640 data=/content/datasets/dataset/data.yaml  
Visualize:       https://netron.app
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11m summary (fused): 303 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs

PyTorch: starting from 'runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 8400) (38.7 MB)

TorchScript: starting export with torch 2.5.1+cu124...
TorchScript: export success ✅ 3.7s, saved as 'runs/detect/train2/weights/best.torchscript' (77.2 MB)

Export complete (4.8s)
Results saved to /content/runs/detect/train

In [12]:
# Validate the model
metrics = model.val(save_json=True)  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

#this will create a new folder with the results of this val (usually train12, train22, train32 etc..), it can be discarded as they are also included in the original training folder

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11m summary (fused): 303 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning /content/datasets/dataset/val/labels.cache... 133 images, 0 backgrounds, 0 corrupt: 100%|██████████| 133/133 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.1GB RAM): 100%|██████████| 133/133 [00:00<00:00, 707.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


                   all        133       5342      0.924      0.783      0.862      0.697
               Bicycle         61         88      0.941       0.75      0.872      0.732
                   Bus         47        120       0.97       0.95      0.975      0.927
                   Car        110       2571      0.963      0.821      0.904      0.761
         Flatbed truck         64         66      0.965      0.909      0.958       0.72
               Minibus         89        318      0.936      0.824      0.899      0.766
            Motorcycle        130        453      0.918      0.714      0.808      0.615
            Pedestrian        106       1036      0.902      0.373      0.563      0.361
              Rickshaw          7          7      0.684      0.857      0.873      0.623
               Scooter         45        135      0.964      0.783      0.893      0.686
                 Truck         66        172      0.991      0.971      0.988      0.927
               Unknow

array([      0.732,     0.92658,      0.7609,     0.71958,     0.76647,     0.61547,      0.3607,      0.6227,     0.68561,     0.92723,     0.38609,     0.85643])

# Download results

In [13]:
from ultralytics.utils.downloads import zip_directory
zipped_res = zip_directory('/content/runs/detect/train2')

Deleting .DS_Store files: []
Deleting __MACOSX files: []


Zipping /content/runs/detect/train2 to /content/runs/detect/train2.zip...: 100%|██████████| 29/29 [00:17<00:00,  1.65file/s]


In [20]:
from shutil import copy2
os.makedirs('/content/drive/MyDrive/Colab Notebooks/models', exist_ok=True)
copy2(zipped_res, os.path.join('/content/drive/MyDrive/Colab Notebooks/models', export_zip_name))

'/content/drive/MyDrive/Colab Notebooks/models haha/trained_model_m.zip'